設定一些東西齁
===

In [3]:
##
from transformers import BertTokenizer
import transformers
import ckip_transformers
from ckiptagger import WS, POS, NER
from ckip_transformers.nlp import CkipPosTagger
from ckip_transformers.nlp import CkipWordSegmenter
from ckip_transformers.nlp import CkipNerChunker

In [7]:
import transformers

In [4]:
tokenizer=BertTokenizer.from_pretrained("bert-base-chinese")

In [5]:
tokens ="[unused1]"
#tokens =['[unused1]']
tokenizer.add_tokens(tokens, special_tokens=True)
tokenizer.save_pretrained('path/to/save/tokenizer')
tokenizer.special_tokens_map


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'additional_special_tokens': ['[unused1]']}

In [6]:
#ws = WS("./data")
#pos = POS("./data")
#ner = NER("./data")
CKIP_POS=CkipPosTagger(model="bert-base",device=-1)
CKIP_WS=CkipWordSegmenter(model="bert-base",device=-1)
CKIP_NER=CkipNerChunker(model="bert-base",device=-1)

文本前處理
===

正式文本
---
<span style="color:red">這邊只有先做1000筆 之後再多做然後寫入txt</span>

In [8]:
with open("語料/原始華語.txt","r",encoding="utf-8") as f:
    data=f.readlines()
full_formal_text=[]
for i in data:
    full_formal_text.append(i[:-1])
full_formal_text[0:5]

['Obama大勝美國首位黑人總統',
 '駐美特派員曹郁芬華府五日報導',
 '歐巴瑪Obama大勝美國首位黑人總統',
 '壓倒性勝利創造新歷史',
 '民主黨總統候選人歐巴瑪四日以壓倒性勝利，']

In [9]:
#64110
#<25 61308
len(full_formal_text)


64110

In [10]:
formal_text=full_formal_text[0:60000]

In [19]:
#ws_results = ws(data[2])
#pos_results = pos(ws_results)
#ner_results = ner(ws_results, pos_results)
formal_ws=CKIP_WS(formal_text)
formal_pos=CKIP_POS(formal_ws)
formal_ner=CKIP_NER(formal_text)

Tokenization:   0%|          | 0/59998 [00:00<?, ?it/s]

Inference:   2%|▏         | 4/235 [03:32<3:24:03, 53.00s/it]


KeyboardInterrupt: 

In [334]:
formal_ws[0]

['Obama', '大勝', '美國', '首位', '黑人', '總統']

In [335]:
formal_ner[0]
#replace

[NerToken(word='Obama', ner='PERSON', idx=(0, 5)),
 NerToken(word='美國', ner='GPE', idx=(7, 9)),
 NerToken(word='首', ner='ORDINAL', idx=(9, 10))]

In [336]:
def filter_by_ner(sent,ner_filter):
    filter_ner_sent=[]
    for j,word in enumerate(sent):
        if word.ner in ner_filter:
            filter_ner_sent.append(word.idx)
            
    return filter_ner_sent

In [337]:
ner_filter=["PERSON"]
formal_ner_person=[filter_by_ner(sent,ner_filter) for sent in formal_ner]
formal_ner_person[0]

[(0, 5)]

In [338]:
count=0
for i in formal_ner_person:
    if i!=[]:
        text_slice=formal_text[count]
        starts, ends = zip(*i)
        start_slices = [int(start) for start in starts]
        end_slices = [int(end) for end in ends]
        person=(f"{i[0][0]}:{i[0][1]}")
        for x in range(len(start_slices)):
            #print(formal_text[count][start_slices[x]:end_slices[x]])
            formal_text[count]=formal_text[count].replace(formal_text[count][start_slices[x]:end_slices[x]],"[unused1]")
            #print(formal_text[count])
    count+=1
formal_noname=formal_text

In [13]:
formal_noname[0]

'Obama大勝美國首位黑人總統'

In [14]:
#數字
formal_tensor=[]
#[PAD]
formal_token=[]

for text in formal_noname:
    tensor=tokenizer.encode(text)
    if len(tensor)<=30:
        tensor_fill=tensor+ [0] * (25 - len(tensor))
        formal_tensor.append(tensor_fill)
        formal_token.append(tokenizer.decode(tensor_fill))
    else:
        print(text)
        formal_noname.remove(text)



   

與 蘇治芬 的 辯護律師 李進勇 、 顧立雄 、 蔡易餘 為了是否 延押 激烈攻防，
於 上個月 二十五日 發表的 「 二OO八年 聯合作戰 環境 評價 報告 」 中 ，


In [15]:
len(formal_tensor)

59996

In [16]:
new_formal_tensor=formal_tensor[:]
for i in new_formal_tensor:
    if len(i)>25:
        formal_tensor.remove(i)

In [17]:
len(formal_tensor)

59962

In [18]:

print(formal_text[0])
print("\n")
print(formal_tensor[0])
print("\n")
print(formal_token[0])

Obama大勝美國首位黑人總統


[101, 100, 1920, 1245, 5401, 1751, 7674, 855, 7946, 782, 5244, 5186, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[CLS] [UNK] 大 勝 美 國 首 位 黑 人 總 統 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [346]:
with open("formal.txt", "w",encoding='UTF-8') as file:
    for item in formal_text:
        file.write(str(item) + "\n")

In [347]:
with open("formal_tensor.txt", "w",encoding='UTF-8') as file:
    for item in formal_tensor:
        file.write(str(item) + "\n")

In [348]:
with open("formal_token.txt", "w",encoding='UTF-8') as file:
    for item in formal_token:
        file.write(str(item) + "\n")

非正式文本
---

In [407]:
import csv
informal_text=[] 
with open("Gossiping-QA-Dataset-2_0.csv", mode="r", encoding="utf-8-sig") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        r=row[1].replace(" ","")
        r=r.replace(",","")
        r=r.replace(".","")
        r=r.replace("。","")
        informal_text.append(r)
       
informal_text[0]

'質感劇本成員都差很多好嗎不要拿腎結石來污辱這群人'

In [408]:
len(informal_text)

774114

In [629]:
informal_text=informal_text[0:80000]
informal_text[0]

'質感劇本成員都差很多好嗎不要拿腎結石來污辱這群人'

In [630]:
informal_ws=CKIP_WS(informal_text)
informal_pos=CKIP_POS(informal_ws)
informal_ner=CKIP_NER(informal_text)

Inference: 100%|██████████| 235/235 [25:22<00:00,  6.48s/it]


In [631]:
ner_filter=["PERSON"]
informal_ner_person=[filter_by_ner(sent,ner_filter) for sent in informal_ner]

In [632]:
count=0
for i in informal_ner_person:
    if i!=[]:
        text_slice=informal_text[count]
        starts, ends = zip(*i)
        start_slices = [int(start) for start in starts]
        end_slices = [int(end) for end in ends]
        person=(f"{i[0][0]}:{i[0][1]}")
        for x in range(len(start_slices)):
            #print(formal_text[count][start_slices[x]:end_slices[x]])
            informal_text[count]=informal_text[count].replace(informal_text[count][start_slices[x]:end_slices[x]],"[unused1]")
            #print(formal_text[count])
    count+=1
informal_noname=informal_text

In [633]:
#數字
informal_tensor=[]
#[PAD]
informal_token=[]

for text in informal_noname:
    tensor=tokenizer.encode(text)
    if len(tensor)<=40:
        tensor_fill=tensor+ [0] * (25 - len(tensor))
        informal_tensor.append(tensor_fill)
        informal_token.append(tokenizer.decode(tensor_fill))
    else:informal_noname.remove(text)

In [634]:
print(informal_tensor[0])
print("\n")
print(informal_token[0])

[101, 6549, 2697, 1206, 3315, 2768, 1519, 6963, 2345, 2523, 1914, 1962, 1621, 679, 6206, 2897, 5575, 5178, 4767, 889, 3738, 6802, 6857, 5408, 782, 102]


[CLS] 質 感 劇 本 成 員 都 差 很 多 好 嗎 不 要 拿 腎 結 石 來 污 辱 這 群 人 [SEP]


In [636]:
new_informal_tensor=informal_tensor[:]
for i in new_informal_tensor:
    if len(i)>25:
        informal_tensor.remove(i)

In [637]:
len(informal_text)

59955

In [639]:
with open("informal.txt", "w",encoding='UTF-8') as file:
    for item in informal_text:
        file.write(str(item) + "\n")

<span style="color:red">生成器</span>
===

BertLMHeadModel 訓練 Model

FORMAL
---

In [10]:
from transformers import AutoTokenizer, AutoModelForMaskedLM,BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('ckiplab/bert-tiny-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese') 

model

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [11]:
with open("formal.txt","r",encoding="utf-8") as f:
    data=f.readlines()
formal_text=[]
for i in data:
    formal_text.append(i[:-1])


In [12]:
f=[]
for i in formal_text:
    x=i.replace(" ",'')
    f.append(x)
    
formal_text=f

In [13]:
len(formal_text)

59963

In [14]:
formal_longer=[]
for i in range(0,len(formal_text)-5,5):
    text=formal_text[i]+formal_text[i+1]+formal_text[i+2]+formal_text[i+3]+formal_text[i+4]
    formal_longer.append(text)

In [15]:
len(formal_longer)

11992

In [16]:
input_ids=tokenizer(formal_longer,return_tensors="pt", padding=True, truncation=True)["input_ids"]

In [17]:
input_ids[0]

tensor([  101,   138,   163, 11316,  8303,  8148,   140,  1920,  1245,  5401,
         1751,  7674,   855,  7946,   782,  5244,  5186,  7688,  5401,  4294,
         3836,  1519,   138,   163, 11316,  8303,  8148,   140,  5836,  2424,
          758,  3189,  1841,  2206,   138,   163,  8171,   138,   163, 11316,
         8303,  8148,   140,   140,   100,  1920,  1245,  5401,  1751,  7674,
          855,  7946,   782,  5244,  5186,  1886,   948,  2595,  1245,  1164,
         1201,  6863,  3173,  3644,  1380,  3696,   712,  7955,  5244,  5186,
          952,  6908,   782,   138,   163, 11316,  8303,  8148,   140,  1724,
         3189,   809,  1886,   948,  2595,  1245,  1164,  8024,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0])

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
num_epochs = 3
for epoch in range(num_epochs):
    dataset = TensorDataset(input_ids)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    for batch in dataloader:
        outputs = model(input_ids=batch[0])
        loss = criterion(outputs.logits.view(-1, model.config.vocab_size), batch[0].view(-1))
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor(5.3970, grad_fn=<NllLossBackward0>)
tensor(4.2658, grad_fn=<NllLossBackward0>)
tensor(3.3929, grad_fn=<NllLossBackward0>)


In [572]:
#model.save_pretrained("formal_model")
#model.save_pretrained("formal_model_large")
#model.save_pretrained("formal_model_large_nospace")
#model.save_pretrained("formal_model_bertLMHead")

In [21]:
#formal_model = BertForMaskedLM.from_pretrained("formal_model")
#formal_model = BertForMaskedLM.from_pretrained("formal_model_large")
#formal_model = BertForMaskedLM.from_pretrained("formal_model_large_nospace")
#formal_model = AutoModelForMaskedLM.from_pretrained("formal_model_large_nospace")
from transformers import BertTokenizer, BertForMaskedLM

formal_model =BertForMaskedLM.from_pretrained("formal_model_bertLMHead")

In [25]:
text=full_formal_text[6]+full_formal_text[7]+full_formal_text[8]+full_formal_text[9]
#text="總統選舉登記結束，確定由藍綠白三方爭霸"
#text="質 感 劇 本 成 員 都 差 很 多 好 嗎 不 要 拿 腎 結 石 來 污 辱 這 群 人"

text=text.replace(" ","")
text

NameError: name 'full_formal_text' is not defined

In [1]:

num_sentences = 1  
#model=AutoModelForMaskedLM.from_pretrained("bert-base-chinese")



for _ in range(num_sentences):
  
    prompt = text
    
  
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
    #input_ids = new_tokenizer(prompt, return_tensors="pt")["input_ids"]
 
    outputs = formal_model.generate(input_ids, max_length=100, num_beams=3, no_repeat_ngram_size=1, do_sample=True, top_k=30,top_p=0.95)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
 
    print("Generated Sentence:", generated_text)


NameError: name 'text' is not defined

informal
---

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM,BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('ckiplab/bert-tiny-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese') 

In [648]:
with open("informal.txt","r",encoding="utf-8") as f:
    data=f.readlines()
informal_text=[]
for i in data:
    informal_text.append(i[:-1])


In [649]:
f=[]
for i in informal_text:
    x=i.replace(" ",'')
    f.append(x)
    
informal_text=f
len(informal_text)

59955

In [650]:
informal_longer=[]
for i in range(0,len(informal_text)-5,5):
    text=informal_text[i]+informal_text[i+1]+informal_text[i+2]+informal_text[i+3]+informal_text[i+4]
    informal_longer.append(text)
len(informal_longer)

11990

In [654]:
input_ids=tokenizer(informal_longer,return_tensors="pt", padding=True, truncation=True)["input_ids"]
input_ids[0]

tensor([  101,  6549,  2697,  1206,  3315,  2768,  1519,  6963,  2345,  2523,
         1914,  1962,  1621,   679,  6206,  2897,  5575,  5178,  4767,   889,
         3738,  6802,  6857,  5408,   782,  1728,  4158,   138,   163, 11316,
         8303,  8148,   140,   679,  3221,  1378,  4124,   782,  8024,  3221,
         3504,  1751,   782,  4265,  1894,  2218,  3221,   138,   163, 11316,
         8303,  8148,   140,  4511,   712,  6235,  3297,  2527,  3647,   749,
         1525,   943,  5143,   679,  5562,  5059,   872,  1558,  3297,   679,
         5562,  5059,  4638,  5080,  1606,  1914,   749,  5503,  2125,  2798,
         4692,  3472,  4413,  5143,  1888,   671,  1831,  5523,  2094,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [652]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

c:\Users\FXZ\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    dataset = TensorDataset(input_ids)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    for batch in dataloader:
        outputs = model(input_ids=batch[0])
        loss = criterion(outputs.logits.view(-1, model.config.vocab_size), batch[0].view(-1))
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [655]:
num_epochs = 3
for epoch in range(num_epochs):
    dataset = TensorDataset(input_ids)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    for batch in dataloader:
        outputs = model(input_ids=batch[0])
        loss = criterion(outputs.logits.view(-1, model.config.vocab_size), batch[0].view(-1))
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

tensor(5.0117, grad_fn=<NllLossBackward0>)
tensor(4.2308, grad_fn=<NllLossBackward0>)
tensor(3.3010, grad_fn=<NllLossBackward0>)
tensor(2.5528, grad_fn=<NllLossBackward0>)
tensor(1.8288, grad_fn=<NllLossBackward0>)
tensor(1.1685, grad_fn=<NllLossBackward0>)
tensor(0.6666, grad_fn=<NllLossBackward0>)
tensor(0.4070, grad_fn=<NllLossBackward0>)
tensor(0.2983, grad_fn=<NllLossBackward0>)
tensor(0.2351, grad_fn=<NllLossBackward0>)
tensor(0.2086, grad_fn=<NllLossBackward0>)
tensor(0.1956, grad_fn=<NllLossBackward0>)
tensor(0.1924, grad_fn=<NllLossBackward0>)
tensor(0.1750, grad_fn=<NllLossBackward0>)
tensor(0.1688, grad_fn=<NllLossBackward0>)
tensor(0.1553, grad_fn=<NllLossBackward0>)
tensor(0.1453, grad_fn=<NllLossBackward0>)
tensor(0.1247, grad_fn=<NllLossBackward0>)
tensor(0.1124, grad_fn=<NllLossBackward0>)
tensor(0.0988, grad_fn=<NllLossBackward0>)
tensor(0.0918, grad_fn=<NllLossBackward0>)
tensor(0.0861, grad_fn=<NllLossBackward0>)
tensor(0.0838, grad_fn=<NllLossBackward0>)
tensor(0.07

In [656]:
model.save_pretrained("informal_model_bertLMHead")

In [657]:
informal_model =BertForMaskedLM.from_pretrained("formal_model_bertLMHead")

cyclegan
===

鑑別器
===

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim


formal_tensor = torch.tensor(formal_tensor)
formal_tensor = formal_tensor.to(torch.float)
input_size = 25

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 128),  
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid() 
        )

    def forward(self, x):
        x = self.fc(x)
        return x

real_text_data = formal_tensor[:1000] 
fake_text_data = torch.randn(1000, input_size)  


discriminator = Discriminator()

criterion = nn.BCELoss()
optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

num_epochs = 15
losses = [] 
for epoch in range(num_epochs):

    outputs_real = discriminator(real_text_data)
    labels_real = torch.ones(outputs_real.size())  
    loss_real = criterion(outputs_real, labels_real)

    outputs_fake = discriminator(fake_text_data)
    labels_fake = torch.zeros(outputs_fake.size()) 
    loss_fake = criterion(outputs_fake, labels_fake)

    total_loss = loss_real + loss_fake
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    losses.append(total_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item()}")


input_text = torch.randn(1, input_size)  
with torch.no_grad():
    output_prob = discriminator(input_text)

print("真假概率:", output_prob.item()) 


<ipython-input-58-43d0b04c7fbb>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  formal_tensor = torch.tensor(formal_tensor)


Epoch [1/15], Loss: 1.1148816347122192
Epoch [2/15], Loss: 0.7904165387153625
Epoch [3/15], Loss: 0.7788472771644592
Epoch [4/15], Loss: 0.7649118900299072
Epoch [5/15], Loss: 0.7498632073402405
Epoch [6/15], Loss: 0.7341395020484924
Epoch [7/15], Loss: 0.7179298996925354
Epoch [8/15], Loss: 0.7731238007545471
Epoch [9/15], Loss: 0.7235013246536255
Epoch [10/15], Loss: 0.6862377524375916
Epoch [11/15], Loss: 0.6775649189949036
Epoch [12/15], Loss: 0.6679651737213135
Epoch [13/15], Loss: 0.6575396656990051
Epoch [14/15], Loss: 0.6463772058486938
Epoch [15/15], Loss: 0.6345552802085876
真假概率: 0.515231728553772


In [60]:
formal_tensor.size()

torch.Size([59962, 25])